In [1]:
%cd /content/drive/MyDrive/CIE 417-Fall23/Lab8

/content/drive/MyDrive/CIE 417-Fall23/Lab8


In [2]:
%ls

auto.csv  Hitters.csv  kernels-and-support-vector-machine-regularization.ipynb  Lab8.ipynb



<h1 align="center">Zewail City of Science and Technology</h1>
<h2 align="center">CIE 417 (Fall 2023)</h2>
<h2 align="center">Lab 8: Feature Engineering and Feature Selection </h2>
<h3 align="center">Eng. Marwa Monier</h3>

---








**Feature Engineering and Feature Selection**


*  **feature extraction and feature engineering**: transformation of raw data into features suitable for modeling;
*  **feature transformation**: transformation of data to improve the accuracy of the algorithm;
* **feature selection**: removing unnecessary features.



---




1-Feature Extraction
each data type needs its feature extraction based on its type:
* Texts
* Images
* Numerical Date
* Time series, web, etc.

2- Feature transformations
*  Normalization and standardization :https://www.geeksforgeeks.org/normalization-vs-standardization/

3- Feature selection

* Sequential Feature Selection
  * ->Forward Stepwise Selection
  * ->Backward Stepwise Selection
*  Statistical approaches
  * -> feature Variance
  * -> feature scoring
    * For regression: r_regression, f_regression, mutual_info_regression
    * For classification: chi2, f_classif, mutual_info_classif
*  Selection by modeling
  * Laso
  * ...etc
  * validation set approach
  * cross validation
     
*  Grid search




---



# Why Feature Engineering is important?

### Example from the MIT lecture with kNN:

> Indented block



Link: https://www.youtube.com/watch?v=h0e2HAPTGF4&t=2362s
> Suppose we want to classify the following three animals as either reptiles or not based on 4 binary features and 1 integer feature. We used the euclidean distance which seprated them good enough; <br>


<img src="https://i.ibb.co/mhThDWY/img1.png" width="450" align='left'> | <img src="https://i.ibb.co/16bxPzB/img2.png" width="450" align='right'>


> As seen above, the distance between boa constrictor and rattlesnake is much less that the distance between any of them and the dart frog. **But, what happens if we added an aligator?**

<img src="https://i.ibb.co/9Y69KG9/img3.png" width="520" align='left'> | <img src="https://i.ibb.co/nD0xPfF/img4.png" width="450" align = 'right'>



# Feature selection


     



---



In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm
import matplotlib.pyplot as plt

## data exploration

In [4]:
df = pd.read_csv('Hitters.csv')
df

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      322 non-null    int64  
 1   Hits       322 non-null    int64  
 2   HmRun      322 non-null    int64  
 3   Runs       322 non-null    int64  
 4   RBI        322 non-null    int64  
 5   Walks      322 non-null    int64  
 6   Years      322 non-null    int64  
 7   CAtBat     322 non-null    int64  
 8   CHits      322 non-null    int64  
 9   CHmRun     322 non-null    int64  
 10  CRuns      322 non-null    int64  
 11  CRBI       322 non-null    int64  
 12  CWalks     322 non-null    int64  
 13  League     322 non-null    object 
 14  Division   322 non-null    object 
 15  PutOuts    322 non-null    int64  
 16  Assists    322 non-null    int64  
 17  Errors     322 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  322 non-null    object 
dtypes: float64

In [6]:
df.describe()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
count,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.00000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,263.000000
mean,380.928571,101.024845,10.770186,50.909938,48.027950,38.742236,7.444099,2648.68323,717.571429,69.490683,358.795031,330.118012,260.239130,288.937888,106.913043,8.040373,535.925882
std,153.404981,46.454741,8.709037,26.024095,26.166895,21.639327,4.926087,2324.20587,654.472627,86.266061,334.105886,333.219617,267.058085,280.704614,136.854876,6.368359,451.118681
min,16.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,19.00000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67.500000
25%,255.250000,64.000000,4.000000,30.250000,28.000000,22.000000,4.000000,816.75000,209.000000,14.000000,100.250000,88.750000,67.250000,109.250000,7.000000,3.000000,190.000000
50%,379.500000,96.000000,8.000000,48.000000,44.000000,35.000000,6.000000,1928.00000,508.000000,37.500000,247.000000,220.500000,170.500000,212.000000,39.500000,6.000000,425.000000
75%,512.000000,137.000000,16.000000,69.000000,64.750000,53.000000,11.000000,3924.25000,1059.250000,90.000000,526.250000,426.250000,339.250000,325.000000,166.000000,11.000000,750.000000
max,687.000000,238.000000,40.000000,130.000000,121.000000,105.000000,24.000000,14053.00000,4256.000000,548.000000,2165.000000,1659.000000,1566.000000,1378.000000,492.000000,32.000000,2460.000000


## Data preperation

In [7]:
# Drop any rows the contain missing values, along with the player names
df = df.dropna()

# Get dummy variables # This a conversion for the categorical to be in
dummies = pd.get_dummies(df[['League', 'Division', 'NewLeague']])

# Extract The target variable #
y = pd.DataFrame(df.Salary)

# Drop the column with the independent variable (Salary), and columns for which we created dummy variables
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')

# Define the feature set X.
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

In [8]:
np.random.seed(seed=12)
train = np.random.choice([True, False], size = len(y), replace = True)
test = np.invert(train)

## Using the sklearn feature selection functions

Notes:
1. The built in sklearn.feature_selection.SequentialFeatureSelector uses cross validation within its algorithm, in addition it needs to specify the number output features.
2. mlxtend library is another option to do feature selection.


### Sequential feature selection (forward / backword)

In [9]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model

In [10]:
knn = KNeighborsClassifier(n_neighbors=3)
lm=linear_model.LinearRegression()
sfs = SequentialFeatureSelector(lm, n_features_to_select=9 ,direction = 'forward' ,cv =10 , scoring = 'neg_mean_squared_error')

In [11]:
X[train].head()


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
6,185.0,37.0,1.0,23.0,8.0,21.0,2.0,214.0,42.0,1.0,30.0,9.0,24.0,76.0,127.0,7.0,1,0,0
9,401.0,92.0,17.0,49.0,66.0,65.0,13.0,5206.0,1332.0,253.0,784.0,890.0,866.0,0.0,0.0,0.0,0,0,0
10,574.0,159.0,21.0,107.0,75.0,59.0,10.0,4631.0,1300.0,90.0,702.0,504.0,488.0,238.0,445.0,22.0,0,0,0
11,202.0,53.0,4.0,31.0,26.0,27.0,9.0,1876.0,467.0,15.0,192.0,186.0,161.0,304.0,45.0,11.0,1,1,1


In [12]:
sfs.fit(X[train], y[train])

SequentialFeatureSelector(cv=10, estimator=LinearRegression(),
                          n_features_to_select=9,
                          scoring='neg_mean_squared_error')

In [13]:
sfs.get_support()

array([False, False,  True,  True,  True,  True, False,  True,  True,
       False,  True, False, False,  True, False, False, False,  True,
       False])

In [14]:
X.columns

Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists',
       'Errors', 'League_N', 'Division_W', 'NewLeague_N'],
      dtype='object')

### Recursive Feature selection

In [15]:
from sklearn.feature_selection import RFECV

In [16]:
selector = RFECV(lm, step=1 , cv  = 10) # step indicate the number of feature to elemintate each time.

In [17]:
selector = selector.fit(X[train], y[train])

In [18]:
selector.support_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True])

#Try it using SVM



---



# Cross Validation for Model Selection
We want to test the performance of a model (or a bunch of models) on our dataset:-
1. **Validation Set Approach** <br>
We usually split our data into two datasets: training and testing. We train our model using the training data and test the performance using the testing data. This is not optimal as while trying different models and improving the performance, we could be creating a model that learns the testing data. Also, the fact that the performance can change depending on how we split our data makes it worse.
> To solve this issue, we can split our data into 3 datasets: training, testing, and validation (60/20/20). The validation dataset is then used while choosing the model and the testing data will only be seen in the final evaluation. **Is this optimal?**
2. **K-fold cross validation**
> In K-fold, the testing data is left as it is, however, the training data is divided into K parts(folds). The model is trained as usual on K-1 of the folds as a training set while one of the folds is held out for validation. This process is repeated untill each fold is used for validation once. The performance measure (accuracy, for example) is the average of the values calculated during the process. This solves the constant training/validation data issue.
<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" width="450" align='center'>

3. **Leave one out cross validation (LOOCV)** <br>
LOO is similar to K-fold CV where K equals the number of samples in the training set. If the dataset has *n* samples then *n-1* samples are used for training while 1 sample is used for validation. This approach can achieve better results, especially if the dataset is small, however, it is computationally expenssive.

<br>

**This method is used for selecting the best model among a bunch of candidate models.**

# Cross Validation

In [19]:
import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt

# Validation Set approch

## Data exploration

In [20]:
auto_data = pd.read_csv('auto.csv', na_values='?').dropna()
auto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cylinders     392 non-null    int64  
 1   displacement  392 non-null    float64
 2   horsepower    392 non-null    float64
 3   weight        392 non-null    float64
 4   acceleration  392 non-null    float64
 5   model_year    392 non-null    int64  
 6   origin        392 non-null    int64  
 7   mpg           392 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 24.6 KB


In [34]:
auto_data.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg
0,8,307.0,130.0,3504.0,12.0,70,1,18.0
1,8,350.0,165.0,3693.0,11.5,70,1,15.0
2,8,318.0,150.0,3436.0,11.0,70,1,18.0
3,8,304.0,150.0,3433.0,12.0,70,1,16.0
4,8,302.0,140.0,3449.0,10.5,70,1,17.0


In [21]:
auto_data.describe()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531,23.445918
std,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518,7.805007
min,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000,9.000000
25%,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000,17.000000
50%,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000,22.750000
75%,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000,29.000000
max,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000,46.600000


## Data preperation and modeling

### Trail 1

In [22]:
train_df = auto_data.sample(196, random_state = 1) # split the data into two halfs # random_state to get the same result every time
val_df = auto_data[~auto_data.isin(train_df)].dropna(how = 'all')

# For simplisity we will use the only parameter horsepower to estimate the mpg
X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_val = val_df['horsepower'].values.reshape(-1,1)
y_val = val_df['mpg']

In [23]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Linear
lm = skl_lm.LinearRegression()

model = lm.fit(X_train, y_train)
print(mean_squared_error(y_val, model.predict(X_val)))

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_val2 = poly.fit_transform(X_val)

model = lm.fit(X_train2, y_train)
print(mean_squared_error(y_val, model.predict(X_val2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_val3 = poly.fit_transform(X_val)

model = lm.fit(X_train3, y_train)
print(mean_squared_error(y_val, model.predict(X_val3)))

23.361902892587224
20.252690858345748
20.325609365972525


### Trail 2

In [24]:
train_df = auto_data.sample(196, random_state = 2) # split the data into two halfs # random_state to get the same result every time
val_df = auto_data[~auto_data.isin(train_df)].dropna(how = 'all')

# For simplisity we will use the only parameter horsepower to estimate the mpg
X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_val = val_df['horsepower'].values.reshape(-1,1)
y_val = val_df['mpg']

model = lm.fit(X_train, y_train)
print(mean_squared_error(y_val, model.predict(X_val)))

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_val2 = poly.fit_transform(X_val)

model = lm.fit(X_train2, y_train)
print(mean_squared_error(y_val, model.predict(X_val2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_val3 = poly.fit_transform(X_val)

model = lm.fit(X_train3, y_train)
print(mean_squared_error(y_val, model.predict(X_val3)))

25.10853905288967
19.722533470492422
19.92136786007267


# Leave-One-Out Cross-Validation (LOOCV)

In [25]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

X = auto_data['horsepower'].values.reshape(-1,1)
y = auto_data['mpg'].values.reshape(-1,1)

# making n_splits = # of entries make the cross_val_score use only one entry for testing and the rest for training (LOOCV)
crossvalidation = KFold(n_splits=392, random_state=None, shuffle=False)

# cross validation scores : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
scores = cross_val_score(lm, X, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)

#print(scores)
print("Folds: " + str(len(scores)) + ", MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))


Folds: 392, MSE: 24.231513517929226, STD: 36.79731503640535


In [26]:
for i in range(1,6):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    scores = cross_val_score(lm, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)

    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))


Degree-1 polynomial MSE: 24.231513517929226, STD: 36.797315036405344
Degree-2 polynomial MSE: 19.248213124489737, STD: 34.99844615178235
Degree-3 polynomial MSE: 19.334984064092605, STD: 35.76513567783445
Degree-4 polynomial MSE: 19.424430307079565, STD: 35.683352752283625
Degree-5 polynomial MSE: 19.033202845369832, STD: 35.31730256892403


# k-Fold Cross-Validation

In [27]:
crossvalidation = KFold(n_splits=10, random_state=None, shuffle=False)

for i in range(1,11):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    scores = cross_val_score(lm, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)

    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 27.439933652339864, STD: 14.510250711281133
Degree-2 polynomial MSE: 21.23584005580157, STD: 11.797327528897458
Degree-3 polynomial MSE: 21.3366061833321, STD: 11.844339714581539
Degree-4 polynomial MSE: 21.353886969306867, STD: 11.986332290218549
Degree-5 polynomial MSE: 20.905594706388634, STD: 12.185475892828933
Degree-6 polynomial MSE: 20.780544653523087, STD: 12.135161876530644
Degree-7 polynomial MSE: 20.953025381629246, STD: 12.059393202127628
Degree-8 polynomial MSE: 21.07710816481234, STD: 12.044551838990557
Degree-9 polynomial MSE: 21.035364254967988, STD: 11.94913957857655
Degree-10 polynomial MSE: 20.978001651203595, STD: 11.798777462329955


# Which model degree will you use ? Why?

## GridSearch CV
After we decide to use a certain model based on the CV scores, we should try to find the best parameters for it to obtain the best performance.

In [28]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

In [29]:


iris = load_iris()
X = iris.data
y = iris.target

In [30]:


scaler = StandardScaler()
X = scaler.fit_transform(X)


In [31]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X, y)

print("mean_squared_error",mean_squared_error(y, clf.predict(X)))
#sorted(clf.cv_results_.keys())

mean_squared_error 0.013333333333333334


In [32]:


C_range = np.logspace(-2, 10, 13)
print(C_range)
gamma_range = np.logspace(-9, 3, 13)

param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X, y)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)
print("mean_squared_error=",mean_squared_error(y, grid.predict(X)))

[1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03 1.e+04 1.e+05 1.e+06 1.e+07
 1.e+08 1.e+09 1.e+10]
The best parameters are {'C': 1.0, 'gamma': 0.1} with a score of 0.97
mean_squared_error= 0.02




---



# Refrence
* https://www.geeksforgeeks.org/normalization-vs-standardization/
* http://rasbt.github.io/mlxtend/user_guide/feature_selection/ExhaustiveFeatureSelector/

* https://ds100.org/course-notes/intro_lec/introduction.html

* https://www.cs.cmu.edu/~kdeng/thesis/feature.pdf
* https://www.kaggle.com/code/kashnitsky/topic-6-feature-engineering-and-feature-selection

* The lab on Cross-Validation is a python adaptation of p. 190-194 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. Written by R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016) and is inspired from http://www.science.smith.edu/~jcrouser/SDS293/labs/lab9-py.html